## Import necessary libraries

In [13]:
import numpy as np # library to handle data in a vectorized manner

import pandas as pd # library for data analsysis
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

import json # library to handle JSON files

!conda install -c conda-forge geopy --yes # uncomment this line if you haven't completed the Foursquare API lab
from geopy.geocoders import Nominatim # convert an address into latitude and longitude values

import requests # library to handle requests
from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe
!pip install lxml

# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors

# import k-means from clustering stage
from sklearn.cluster import KMeans

#!conda install -c conda-forge folium=0.5.0 --yes # uncomment this line if you haven't completed the Foursquare API lab
import folium # map rendering library

print('Libraries imported.')

Solving environment: done


==> WARNING: A newer version of conda exists. <==
  current version: 4.5.11
  latest version: 4.8.0

Please update conda by running

    $ conda update -n base -c defaults conda



# All requested packages already installed.

     |████████████████████████████████| 5.8MB 16.2MB/s eta 0:00:01
Libraries imported.


### Download and explore data

In [53]:
toronto_data= pd.DataFrame(pd.read_html('https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M')[0])
toronto_data.shape

(287, 3)

### Let's look at the data and update the name of the first column

In [55]:
toronto_data.rename(columns={"Postcode": "PostalCode"},inplace= True)
toronto_data.head()

,PostalCode,Borough,Neighborhood
0,M1A,Not assigned,Not assigned
1,M2A,Not assigned,Not assigned
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,Harbourfront


### Let's drop all the rows that have no borough assigned

In [96]:
toronto_data = toronto_data[toronto_data['Borough'] != 'Not assigned']
toronto_data.head()

,PostalCode,Borough,Neighborhood
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,Harbourfront
5,M6A,North York,Lawrence Heights
6,M6A,North York,Lawrence Manor


### Aggregate postal codes in only one row

In [97]:
g_toronto_data = toronto_data.groupby(['PostalCode'], as_index = False).agg({'Borough':'first','Neighborhood':', '.join})
g_toronto_data.head()

,PostalCode,Borough,Neighborhood
0,M1B,Scarborough,"Rouge, Malvern"
1,M1C,Scarborough,"Highland Creek, Rouge Hill, Port Union"
2,M1E,Scarborough,"Guildwood, Morningside, West Hill"
3,M1G,Scarborough,Woburn
4,M1H,Scarborough,Cedarbrae


### Replace name of the Neighborhood with name of the Borough if not assigned

In [103]:
g_toronto_data['Neighborhood'] = np.where(g_toronto_data['Neighborhood'] == 'Not assigned', g_toronto_data['Borough'],g_toronto_data['Neighborhood'])
g_toronto_data.head()

,PostalCode,Borough,Neighborhood
0,M1B,Scarborough,"Rouge, Malvern"
1,M1C,Scarborough,"Highland Creek, Rouge Hill, Port Union"
2,M1E,Scarborough,"Guildwood, Morningside, West Hill"
3,M1G,Scarborough,Woburn
4,M1H,Scarborough,Cedarbrae


In [104]:
g_toronto_data.shape

(103, 3)